# Part 1
## Task 1

In [74]:
import pandas as pd

user_clip = pd.read_csv('user_clip.csv').dropna()
r_avg = user_clip['weight'].mean()
users_bias = user_clip.groupby('user_id')['weight'].mean() - r_avg
clips_bias = user_clip.groupby('clip_id')['weight'].mean() - r_avg

In [75]:
test_df = pd.read_csv('test.csv').filter(['user_id', 'clip_id']).dropna()
users_bias = users_bias.reset_index().rename(columns={'index': 'user_id', 'weight': 'user_bias'})
clips_bias = clips_bias.reset_index().rename(columns={'index': 'clip_id', 'weight': 'clip_bias'})
test_df = test_df.merge(users_bias, on=['user_id'], how='left')
test_df = test_df.merge(clips_bias, on=['clip_id'], how='left')
test_df['prediction'] = r_avg + test_df['user_bias'] + test_df['clip_bias']
test_df.filter(['user_id', 'clip_id', 'prediction']).rename(columns={'prediction': 'weight'}).to_csv('319044434_314779166_task1.csv', index=False)

In [76]:
user_clip = user_clip.merge(users_bias, on=['user_id'], how='left')
user_clip = user_clip.merge(clips_bias, on=['clip_id'], how='left')
user_clip['prediction'] = r_avg + user_clip['user_bias'] + user_clip['clip_bias']
user_clip['prediction'] = user_clip['prediction'].clip(lower=0)

In [77]:
def f1(df):
    error = ((df['prediction'] - df['weight']) ** 2).sum()
    regularization = 0.1 * ((df['user_bias'] ** 2).sum() + (df['clip_bias'] ** 2).sum())
    return error + regularization

print(f1(user_clip))

# TODO: Question for the metargel:
# 1. What do we do with predictions below 0 (both in train and test)?
# 2. What do we do with missing values (currently - we drop them)?
# 3. Is f1 on the test or train?

965092157923.618


## Task 2